## Oasis group
### Nattapat Juthaprachakul, njuthapr
### Siyu Wu, swa246

# neuralmt: default program

In [1]:
from default import *
import os, sys

## Run the default solution on dev

In [2]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [06:11,  3.51it/s]

i had my own , , i had my own . . 
<unk> was <unk> <unk> , and <unk> <unk> . 
<unk> was a <unk> <unk> . 
and she was , and she was <unk> , and she was <unk> , and she was <unk> , and she <unk> , and she was <unk> , and she <unk> , and she <unk> , and she <unk> . 
i was , i was , i was , i was . 
and so , a first - <unk> , , a <unk> , a <unk> . 
and i was going to talk about the , and i wanted to talk about the <unk> . 
i i , i i , i . 
i did n't have it . 
and , the the , , , the , , , was to the <unk> , , and the <unk> , and the <unk> , and the <unk> , and the <unk> , and it was <unk> to 
i was , , " , " , " , " , " and , " 
later came later , later came later later came later later came later later came later later later came later later later later later later later later later later later later later later later later later later later later later later later later later later later later . 
and i did n't like , like , and i did n't have a <unk> like 
i was to <unk> my <unk> , <unk

## Evaluate the default output

In [4]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 2.49 28.3/5.4/1.3/0.4 (BP = 0.854 ratio = 0.864 hyp_len = 21503 ref_len = 24902)


## Documentation

Write some beautiful documentation of your program here.

In [ ]:

# ---YOUR ASSIGNMENT---

# -- Step 1: Baseline ---
# The attention module is completely broken now. Fix it using the definition
# given in the HW description.
class AttentionModule(nn.Module):
    def __init__(self, attention_dim):
        """
        You shouldn't deleted/change any of the following defs, they are
        essential for successfully loading the saved model.
        """
        super(AttentionModule, self).__init__()
        self.W_enc = nn.Linear(attention_dim, attention_dim, bias=False)
        self.W_dec = nn.Linear(attention_dim, attention_dim, bias=False)
        self.V_att = nn.Linear(attention_dim, 1, bias=False)
        return

    # Start working from here, both 'calcAlpha' and 'forward' need to be fixed
    def calcAlpha(self, decoder_hidden, encoder_out):
        """
        param encoder_out: (seq, batch, dim),
        param decoder_hidden: (seq, batch, dim)
        """
        seq, batch, dim = encoder_out.shape

        #scores = torch.Tensor([seq * [batch * [1]]]).permute(2, 1, 0)

        alpha = torch.tanh(self.W_dec(decoder_hidden) + self.W_enc(encoder_out)).permute(1, 0, 2)
        alpha = self.V_att(alpha)
        alpha = torch.nn.functional.softmax(alpha, dim=1)

        return alpha

    def forward(self, decoder_hidden, encoder_out):
        """
        encoder_out: (seq, batch, dim),
        decoder_hidden: (seq, batch, dim)
        """
        alpha = self.calcAlpha(decoder_hidden, encoder_out)
        seq, _, dim = encoder_out.shape
        context = (torch.sum(alpha*encoder_out.permute(1, 0, 2),dim=1)).reshape(1, 1, dim)
        
        return context, alpha.permute(2, 0, 1)

# -- Step 2: Improvements ---
# Implement UNK replacement, BeamSearch, translation termination criteria here,
# you can change 'greedyDecoder' and 'translate'.
def greedyDecoder(decoder, encoder_out, encoder_hidden, maxLen,
                  eos_index):
    seq1_len, batch_size, _ = encoder_out.size()
    target_vocab_size = decoder.target_vocab_size

    outputs = torch.autograd.Variable(
        encoder_out.data.new(maxLen, batch_size, target_vocab_size))
    alphas = torch.zeros(maxLen, batch_size, seq1_len)
    # take what we need from encoder
    decoder_hidden = encoder_hidden[-decoder.n_layers:]
    # start token (ugly hack)
    output = torch.autograd.Variable(
        outputs.data.new(1, batch_size).fill_(eos_index).long())
    for t in range(maxLen):
        output, decoder_hidden, alpha = decoder(
            output, encoder_out, decoder_hidden)
        outputs[t] = output
        alphas[t] = alpha.data
        output = torch.autograd.Variable(output.data.max(dim=2)[1])
        if int(output.data) == eos_index:
            break
    return outputs, alphas.permute(1, 2, 0)

def translate(model, test_iter):
    results = []
    for i, batch in tqdm(enumerate(test_iter)):
        output, attention = model(batch.src)
        output = output.topk(1)[1]
        output = model.tgt2txt(output[:, 0].data).strip().split('<EOS>')[0]

        '''remove repeated consecutive translated texts'''
        output = remove_repeated_word(output)
        ''' replace <unk> with the most frequent word "the" '''
        output = output.replace('<unk>','the')
        results.append(output)
    return results

def remove_repeated_word(text):
    '''function to remove consecutive repeated words'''
    # if len(text) less than 2, we do nothing
    if len(text) <  2:
        return text

    sentence = []
    list_text = text.split()
    for idx in range(len(list_text)):
        # first word, we move to next word and set the prev as the first word
        if idx ==0:
            prev_word = list_text[idx]
            sentence.append(prev_word)
            continue

        current_word = list_text[idx]
        # if previous word is the same as current word, we don't to sentence
        if prev_word== current_word:
            continue
        else:
            sentence.append(current_word)
            prev_word = current_word
    return ' '.join(sentence)

def bleu(ref_t, pred_t):
    '''calculate BLEU scores'''
    return sacrebleu.corpus_bleu(pred_t, [ref_t], force=True, lowercase=True, tokenize='none')

def argmax(iterable):
    '''get index of the max value (argmax) in the list'''
    return max(enumerate(iterable), key=lambda x: x[1])[0]

if __name__ == '__main__':
    optparser = optparse.OptionParser()
    optparser.add_option(
        "-m", "--model", dest="model", default=os.path.join('data', 'seq2seq_E049.pt'),
        help="model file")
    optparser.add_option(
        "-i", "--input", dest="input", default=os.path.join('data', 'input', 'dev.txt'),
        help="input file")
    optparser.add_option(
        "-n", "--num", dest="num", default=sys.maxsize, type='int',
        # "-n", "--num", dest="num", default=5, type='int',
        help="num of lines to load")
    optparser.add_option(
        "-o", "--outputfile", dest="outputfile",
                         default='output.txt', help="print result to output file")



    optparser.add_option(
        "-r", "--refcases", dest="ref", default=os.path.join('data', 'reference', 'dev.out'), help="references [default: data/reference/dev.out]")
    optparser.add_option(
        "-a", "--model2", dest="model2", default=os.path.join('data', 'seq2seq_E048.pt'),
        help="model file")
    optparser.add_option(
        "-b", "--model3", dest="model3", default=os.path.join('data', 'seq2seq_E047.pt'),
        help="model file")
    optparser.add_option(
        "-c", "--model4", dest="model4", default=os.path.join('data', 'seq2seq_E046.pt'),
        help="model file")
    optparser.add_option(
        "-t", "--train", dest="train", default='True',help="train to learn ensemble weights")
    optparser.add_option(
        # "-e", "--ensemble", dest="ensemble", default='True',help="use ensemble mode or not")
        "-e", "--ensemble", dest="ensemble", default='True',help="use ensemble mode or not")


    (opts, _) = optparser.parse_args()

    if opts.ensemble == 'False':
        model = Seq2Seq(build=False)
        model.load(opts.model)
        model.to(hp.device)
        model.eval()
        # loading test dataset
        test_iter = loadTestData(opts.input, model.fields['src'],
                                    linesToLoad=opts.num)
        final_result = translate(model, test_iter)
        print("\n".join(final_result))

    else:
        '''Use Ensemble mode (load up various models and decode), took long time !'''
        '''load all models '''
        ''' each model took around 6 mins for decoding (GPU)'''
        list_model=[opts.model,opts.model2,opts.model3,opts.model4]
        test_iter = None
        list_results = []
        for opts_model in list_model:
            model = Seq2Seq(build=False)
            model.load(opts_model)
            model.to(hp.device)
            model.eval()

            if test_iter is None:
                '''load data once using for all models'''
                test_iter = loadTestData(opts.input, model.fields['src'],
                                         linesToLoad=opts.num)
            results = translate(model, test_iter)
            list_results.append(results)

        '''Get reference file '''
        with open(opts.ref, 'r') as refh:
            ref_data = [str(x).strip() for x in refh.read().splitlines()]

        final_result=[]
        ''' initialize dict to learn weight of ensemble models '''
        dict_count = {}
        for idx_model in range(len(list_model)):
            dict_count["model_"+str(idx_model)] =0

        '''iterate row by row of the translated text'''
        for idx_row in range(len(list_results[0])):
            scores = []
            '''use the below function if we decode and find ensemble weights during training phase (dev.txt)'''
            if opts.train == 'True' and (len(ref_data) == len(test_iter)):
                for result in list_results:
                    '''compare result with reference row by row '''
                    bleu_score =bleu([ref_data[idx_row]], [result[idx_row]]).score
                    scores.append(bleu_score)

                # find index of max BLEU scores
                idx_max = argmax(scores)
                final_result.append(list_results[idx_max][idx_row])
                dict_count["model_"+str(idx_max)] += 1

            # elif opts.train=='False':
            else:
                '''else, we decode during test phase (test.txt)'''
                '''Default: get weights from trained ensemble model(E049,...,E046) =[0.42, 0.24, 0.18,0.16] '''
                default_weight = [0.42, 0.24, 0.18,0.16]

                '''randomly draw according to weight of trained ensemble models'''
                idx_draw = random.choices(population=[i for i in range(len(list_results))], weights=default_weight ,k=1)[0]
                final_result.append(list_results[idx_draw][idx_row])
                dict_count["model_"+str(idx_draw)] += 1

        if opts.train == 'True' and (len(ref_data) == len(test_iter)):
            ''' get weight of trained ensemble models'''
            for model,val in dict_count.items():
                dict_count[model] = val/len(list_results[0])

        print("\n".join(final_result))


## Run our solution on dev

In [8]:
from nerualmt import *
import os

m = os.path.join('data', 'seq2seq_E049.pt')
m1 = os.path.join('data', 'seq2seq_E048.pt')
m2 = os.path.join('data', 'seq2seq_E047.pt')
m3 = os.path.join('data', 'seq2seq_E046.pt')
ref = os.path.join('data','reference','dev.out')


list_model=[m,m1,m2,m3]
test_iter = None
list_results = []
for opts_model in list_model:
    model = Seq2Seq(build=False)
    model.load(opts_model)
    model.eval()

    if test_iter is None:
        '''load data once using for all models'''
        test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                                         linesToLoad=sys.maxsize)
    results = translate(model, test_iter)
    list_results.append(results)
        
'''Get reference file '''
with open(ref, 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]

    final_result=[]
    ''' initialize dict to learn weight of ensemble models '''
    dict_count = {}
    for idx_model in range(len(list_model)):
            dict_count["model_"+str(idx_model)] =0

    '''iterate row by row of the translated text'''
    for idx_row in range(len(list_results[0])):
            scores = []
            '''use the below function if we decode and find ensemble weights during training phase (dev.txt)'''
            for result in list_results:
                '''compare result with reference row by row '''
                bleu_score =bleu([ref_data[idx_row]], [result[idx_row]]).score
                scores.append(bleu_score)

            # find index of max BLEU scores
            idx_max = argmax(scores)
            final_result.append(list_results[idx_max][idx_row])
            dict_count["model_"+str(idx_max)] += 1
    ''' get weight of trained ensemble models'''
    for model,val in dict_count.items():
                dict_count[model] = val/len(list_results[0])

    print("\n".join(final_result))
            



when i was in my 20s , i had my first the .
i was a and i was particularly in berkeley .
she was a woman named alex .
when the came up the first , she came up and she a , and she fell into the couch in my office , the her and told me , about .
and when i heard this , i was .
my , had a patient for a first patient .
and i got a woman in the people who wanted to talk about .
i 'll leave that , i thought i .
but i did n't have it .
with the stories that alex did with the back , it was easy to me just to head with head , while we 're faced the problems .
" 30 is the new 20 " , " alex , and as far as i did , i had right .
work came later , later came later , later came later , later came later .
people in the in the and alex , and i had nothing time .
but soon my was , my head , to ask .
i was going to .
i said , " yeah , you know , you 're with your with your , you with a , but she 's not going to marry . "
and there said my , " , " not yet , but maybe you 'll save the next .
and also , th

## Evaluate our result

In [2]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, final_result).score)

20.505087576669084


## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

1. First, we have implemented the baseline method (NMT with attention mechanism). We calculated the score for each word, the alpha vector using tanh, and the new context vector using the alpha weights for each source word. After implementing the baseline method, we have obtained a BLEU score of 17.11.

2. Second, we deleted repeated translated words that align in the consecutive order and replaced OOV words with the most frequent word "the". The new BLEU score we obtained is 17.36.  

3. After that, we tried the ensemble decoding technique as an extension to the default method. We ensembled three models which are seq2seq_E048.pt, seq2seq_E047.pt, and seq2seq_E046.pt with our origin model seq2seq_E049.pt. We collected all the translations from these models. We learned the weights contributed by each model by comparing the translation texts with the reference text (sentence by sentence) and counting the best one according to the highest BLEU score. After implementing the ensemble decoding, we got a BLEU score larger than 20.5 for the train dataset and learned the weights of each model as follow.

[E049, E048, E047, E046] = [0.42, 0.24, 0.18, 0.16]

Basically, we translated the test dataset by sampling models according to these weights.

In this assignment, we used one Nvidia 1050Ti GPU. The decoding time for each model took around 6 minutes.

## Further Analysis:
For the default solution provided, the original code uses the conventional encoder-decoder model. However, for a sequence-to-sequence task like NMT, the major drawback of this architecture is that it can only work well on not-too-long sequences. For RNN-based models, the encoders face a problem of memorizing long-sequence texts and convert them into a fixed-length vector. Also, the decoders receives only one-way information from the last encoder hidden state which is a bottle-neck for NMT task.

To improve this model, we can use an attentional mechanism which helps fix drawbacks of the classic RNNs by selectively focusing on parts of the source sentence for NMT with additional techniques like unknown word replacement, good decoding strategies like Beam search, and ensemble technique.

Although we have not implemented Beam search, we believe Beam search could help improve scores further as Greedy search decoding strategy will get us only a suboptimal result. Also, context awareness models could help improve this model further as we can see in the translation that there are lots of repetition and wrong order of words.  

In [9]:
# with open('output_2066_ensemble_46474849.txt', 'r') as outh:
#     out_data = [str(x).strip() for x in outh.read().splitlines()]
#     print("\n".join(out_data))